#### Imports

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons, make_circles

In [3]:
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [4]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset

#### Create datasets

In [5]:
nTot = 1000
circles_data, circles_labels = make_circles(n_samples = nTot)

In [6]:
# idx_0 = np.where(circles_labels==0)
# idx_1 = np.where(circles_labels==1)
# plt.scatter(circles_data[idx_0,0],circles_data[idx_0,1])
# plt.scatter(circles_data[idx_1,0],circles_data[idx_1,1])
# plt.axis('equal')
# plt.show()

#### Split datat into train and test

In [7]:
fracTrain = 0.8
fracTest = 1 - fracTrain

nTrain = int(nTot*fracTrain)
nTest = nTot - nTrain

In [8]:
nRounds = 3

In [9]:
repsTrain_solo = circles_data[:nTrain]
repsTest_solo = circles_data[nTrain:]

labelsTrain = circles_labels[:nTrain].astype(float)
labelsTest = circles_labels[nTrain:].astype(float)

In [10]:
dimProj = 10
A = np.random.rand(2,dimProj)

repsTrain_proj = repsTrain_solo@A
repsTest_proj = repsTest_solo@A

#### Make sequences

In [11]:
def makeSequences(dataset,Trounds=1):
    if Trounds==1:
        return np.expand_dims(dataset,1)
    elif Trounds>1:
        numSamps, numFeats = dataset.shape
        dataset_sequential = np.zeros((numSamps,Trounds,numFeats))
        for kk in range(Trounds):
            dataset_sequential[:,kk,:] = dataset
        return dataset_sequential

In [12]:
repsTrain = repsTrain_proj
repsTest = repsTest_proj

#### Convert data to tensors

In [13]:
dataTrain = Variable(torch.from_numpy(repsTrain)).requires_grad_(True)
yTrain = Variable(torch.from_numpy(labelsTrain)).requires_grad_(True)

dataTest = Variable(torch.from_numpy(repsTest)).requires_grad_(True)
yTest = Variable(torch.from_numpy(labelsTest)).requires_grad_(True)

#### Train and test loaders

In [14]:
## create dataset and dataloader
tensorTrainData = TensorDataset(dataTrain,yTrain)
tensorTestData = TensorDataset(dataTest,yTest)

bs = 128 ## batch size
train_loader = DataLoader(tensorTrainData, batch_size=bs, shuffle=True)
test_loader = DataLoader(tensorTestData, batch_size=bs, shuffle=False)

#### Test for CUDA

In [15]:
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('No GPU, training on CPU')
else:
    print('GPU found, training on GPU')

No GPU, training on CPU


#### RNN

In [16]:
class RNNpredictor(nn.Module):
    def __init__(self, seq_len=3, n_features=10, hidden_dim=5):
        super(RNNpredictor, self).__init__()
        
        ##Encoder
        self.layer1 = nn.RNN(input_size = n_features, hidden_size = hidden_dim,
                                 num_layers = 1, batch_first = True, bidirectional = False)
        
        ## FC layer
        self.fc = nn.Linear(hidden_dim, 2)
        
    def forward(self, x):
        
        ## layer 1
        x, lastHidden = self.layer1(x)
        
        classifyOp = self.fc(lastHidden)
        
        return classifyOp

#create the NN
model = RNNpredictor()
print(model)

#move tensors to GPU if available
if train_on_gpu:
    model.cuda()

RNNpredictor(
  (layer1): RNN(10, 5, batch_first=True)
  (fc): Linear(in_features=5, out_features=2, bias=True)
)


In [31]:
import numpy as np
import torch
from torch import nn
from torch.nn import ReLU
from torch.nn import Tanh
from abc import ABC, abstractmethod

input_dim = 10
hidden_dim = 2
output_dim = 1

class RNNConfiguration:
    """
    Storage class for configuring RNNs
    """
    def __init__(self):
        """ Default configuration """
        # A matrix representing the input weights
        # These should be randomly initialized.
        self.input_weights = np.random.rand(hidden_dim, input_dim)
        self.input_bias = np.random.rand(hidden_dim)
        # A matrix representing the recurrent weights
        self.recurrent_weights = np.random.rand(hidden_dim, hidden_dim)
        self.recurrent_bias = np.random.rand(hidden_dim)
        # A matrix representing the output weights
        self.output_weights = np.ones((output_dim, hidden_dim))
        self.output_bias = np.ones((output_dim))
        self.T = 2
        self.activation = ReLU()

def make_parameter(matrix, grad=False):
    return nn.Parameter(torch.tensor(matrix, dtype=torch.double, requires_grad=grad))

class UpdatePolicy(ABC):
    """ Abstract base class for updating a weight given a firing history. """
    @abstractmethod
    def update(self, firing_history):
        """ Should return weight update given firing history. """
        pass

class HebbianRule(UpdatePolicy):
    """
        An update policy that manually implements a Hebb's rule type update,
        that increases a weight based on the covariance between their firing
        histories.
    """
    def __init__(self, step_size):
        self.step_size = step_size

    def update(self, firing_history):
        covar = sum(x * y for x, y in zip(*firing_history))
        variances = [sum(x ** 2 for x in history) for history in firing_history]
        return covar / sum(variances) * self.step_size


class UpdateNetwork(nn.Module, UpdatePolicy):
    """
        A small neural network used to update the network weights.
        Takes as input the outputs of each neuron in the network, and outputs
        deltas for the weights.
    """
    def __init__(self, node_number, weight_number, hidden_size): 
        super(UpdateNetwork, self).__init__()
        # Add 1 to the input size to make room for the bias,
        self.layer1 = make_parameter(np.ones((hidden_size, node_number+1))/100, True)
        self.layer2 = make_parameter(np.ones((weight_number, hidden_size))/100, True)
        self.activation = Tanh()

    def update(self, firing_history):
        concat_history = torch.cat((firing_history,torch.tensor([1])), 0)
        y = self.activation(self.layer1 @ concat_history)
        return self.activation(self.layer2 @ y)

class RNNSimple(nn.Module):
    """ A simple recurrrent neural network """
    def __init__(self, config): 
        super(RNNSimple, self).__init__()
        # A matrix representing the input weights
        self.input_weights = torch.tensor(config.input_weights)
        self.input_bias = torch.tensor(config.input_bias)
        # A matrix representing the recurrent weights
        self.recurrent_weights = torch.tensor(config.recurrent_weights)
        self.recurrent_bias = torch.tensor(config.recurrent_bias)
        # A matrix representing the output weights
        self.output_weights = torch.tensor(config.output_weights)
        self.output_bias = torch.tensor(config.output_bias)
        # Number of rounds of recurrent activity
        self.T = config.T
        # ACtivation function
        self.activation = config.activation

    def forward(self, x):
        return self._forward(self.recurrent_weights, x)

    def _forward(self, recurrent_weights, x):
        y = self.activation(self.input_weights @ x + self.input_bias)
        intermediate = [y]
        for i in range(self.T):
            intermediate.append(self.activation(recurrent_weights @ intermediate[-1] + self.recurrent_bias))
        return self.activation(self.output_weights @ intermediate[-1] + self.output_bias), intermediate

T = 2
c = RNNConfiguration()
c.T = T
m = RNNSimple(c)
updater = UpdateNetwork(hidden_dim * (T+1), hidden_dim * hidden_dim, 5)

updater.zero_grad()
optimizer = torch.optim.SGD(updater.parameters(), lr=0.001)
for x, label in zip(dataTrain, yTrain):
    optimizer.zero_grad()
    output, firing = m.forward(x)
    firing_history = torch.cat(firing, 0)
    # Update recurrent weights
    update = torch.reshape(updater.update(firing_history), (hidden_dim, hidden_dim))
    new_weights = m.recurrent_weights + update
    loss = ((label - m._forward(new_weights, x)[0])**2)
    #loss = updater.layer1[0,0]**2

    loss.backward()
    print("gradient: ", updater.layer1.grad)
    optimizer.step()

    m.recurrent_weights = torch.tensor(new_weights)


gradient:  tensor([[1.5389, 1.0977, 2.6796, 1.6820, 3.8350, 2.0532, 1.2550],
        [1.5389, 1.0977, 2.6796, 1.6820, 3.8350, 2.0532, 1.2550],
        [1.5389, 1.0977, 2.6796, 1.6820, 3.8350, 2.0532, 1.2550],
        [1.5389, 1.0977, 2.6796, 1.6820, 3.8350, 2.0532, 1.2550],
        [1.5389, 1.0977, 2.6796, 1.6820, 3.8350, 2.0532, 1.2550]],
       dtype=torch.float64)
gradient:  tensor([[2.5540, 2.3546, 4.4124, 2.3297, 5.4730, 2.6437, 1.3275],
        [2.5540, 2.3546, 4.4124, 2.3297, 5.4730, 2.6437, 1.3275],
        [2.5540, 2.3546, 4.4124, 2.3297, 5.4730, 2.6437, 1.3275],
        [2.5540, 2.3546, 4.4124, 2.3297, 5.4730, 2.6437, 1.3275],
        [2.5540, 2.3546, 4.4124, 2.3297, 5.4730, 2.6437, 1.3275]],
       dtype=torch.float64)
gradient:  tensor([[0.0000, 0.0000, 0.0026, 0.0033, 0.0070, 0.0048, 0.0038],
        [0.0000, 0.0000, 0.0026, 0.0033, 0.0070, 0.0048, 0.0038],
        [0.0000, 0.0000, 0.0026, 0.0033, 0.0070, 0.0048, 0.0038],
        [0.0000, 0.0000, 0.0026, 0.0033, 0.0070, 0.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


gradient:  tensor([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]], dtype=torch.float64)
gradient:  tensor([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]], dtype=torch.float64)
gradient:  tensor([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]], dtype=torch.float64)
gradient:  tensor([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]], dtype=torch.float64)
gradient:  tensor([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 

In [ ]:
m

tensor([[-27.7263, -24.6118],
        [-27.8107, -24.7317]], dtype=torch.float64)

In [38]:
for x, label in zip(dataTest, yTest):
  model_y = m.forward(x)
  print(label - model_y[0])

tensor([-1.], dtype=torch.float64, grad_fn=<SubBackward0>)
tensor([-2.5485], dtype=torch.float64, grad_fn=<SubBackward0>)
tensor([-1.5485], dtype=torch.float64, grad_fn=<SubBackward0>)
tensor([-1.5485], dtype=torch.float64, grad_fn=<SubBackward0>)
tensor([-2.5485], dtype=torch.float64, grad_fn=<SubBackward0>)
tensor([-1.5485], dtype=torch.float64, grad_fn=<SubBackward0>)
tensor([-2.5485], dtype=torch.float64, grad_fn=<SubBackward0>)
tensor([-1.5485], dtype=torch.float64, grad_fn=<SubBackward0>)
tensor([-1.], dtype=torch.float64, grad_fn=<SubBackward0>)
tensor([-1.5485], dtype=torch.float64, grad_fn=<SubBackward0>)
tensor([-1.5485], dtype=torch.float64, grad_fn=<SubBackward0>)
tensor([-1.5485], dtype=torch.float64, grad_fn=<SubBackward0>)
tensor([-1.], dtype=torch.float64, grad_fn=<SubBackward0>)
tensor([-2.5485], dtype=torch.float64, grad_fn=<SubBackward0>)
tensor([-1.], dtype=torch.float64, grad_fn=<SubBackward0>)
tensor([-1.5485], dtype=torch.float64, grad_fn=<SubBackward0>)
tensor([

In [ ]:

torch.cat((x, x, x), 0)

tensor([-0.4562, -0.3986, -0.7520, -0.5104, -0.4790, -1.2215, -0.4153, -0.1850,
        -0.2501, -0.7418, -0.4562, -0.3986, -0.7520, -0.5104, -0.4790, -1.2215,
        -0.4153, -0.1850, -0.2501, -0.7418, -0.4562, -0.3986, -0.7520, -0.5104,
        -0.4790, -1.2215, -0.4153, -0.1850, -0.2501, -0.7418],
       dtype=torch.float64, grad_fn=<CatBackward>)

#### Count number of parameters

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(model)

97

#### Trial pass

In [ ]:
for data, target in train_loader:
    if train_on_gpu:
        data, target = data.float().cuda(), target.float().cuda()
        op = model(data)

print(data.shape)
print(op.shape)

torch.Size([32, 10])


NameError: ignored

#### Hebbian update - MLP mapping

#### MLP Architecture

In [ ]:
class MLPAE(nn.Module):
    def __init__(self):
        super(MLPAE, self).__init__()
        
        ##Encoder
        self.layer1 = nn.Linear(10, 5)
        self.layer2 = nn.Linear(5,2)
        self.layer3 = nn.Linear(2, 5)
        self.layer4 = nn.Linear(5,10)
        
        self.dropout = nn.Dropout(0.3)
        
    def forward(self, x):
        
        ## layer 1
        x = F.relu(self.layer1(x))
        x = self.dropout(x)
        x = F.relu(self.layer2(x))
        x_latent = self.dropout(x)
        x = F.relu(self.layer3(x))
        x = self.dropout(x)
        x_recon = self.layer4(x)
        
        return x_recon, x_latent

#create the NN
model = MLPAE()
print(model)

#move tensors to GPU if available
if train_on_gpu:
    model.cuda()